# Creación de Identificadores para Documentos Legales

Este notebook genera identificadores únicos (hashes) y hace matching entre archivos y catálogos:

## Proceso:

### 1. Procesamiento de Catálogo de Leyes y Reglamentos
- Carga el catálogo completo de leyes (`leyes.csv`)
- Limpia acentos de los nombres
- Crea hashes MD5 únicos de 8 caracteres (`doc_id`)
- Guarda el catálogo con IDs en `leyes_hash.csv`

### 2. Procesamiento de Artículos Extraídos
- Carga los artículos extraídos de `extracted_articles.xlsx`
- Hace **fuzzy matching** entre nombres de archivo y el catálogo de leyes
- Agrega `Nombre_Documento` y `doc_id` a cada artículo
- Crea `art_id` único combinando `doc_id` + número de artículo
- Guarda resultado en `legal_documents_with_ids.csv`

### 3. Procesamiento de Entes Públicos
- Carga catálogo de entes públicos
- Crea hashes únicos (`entity_id`)

El fuzzy matching usa un threshold de 90% y normaliza nombres (quita `%`, convierte a minúsculas) para manejar variaciones.


In [21]:
# Importar librerías
import pandas as pd
import re
import sys
import os
from urllib.parse import unquote
from rapidfuzz import fuzz, process

# Importar funciones desde el módulo de funciones
sys.path.append('../functions')
from hash_functions import (
    create_document_hash,
    normalize_text_for_hash,
    clean_section_title,
    remove_accents
)


In [22]:
def crear_mapeo_nombres(df_leyes):
    """
    Crea un diccionario que mapea nombres de archivo a información de documentos legales.
    Extrae el nombre del archivo de las columnas link_docx y link_pdf.
    
    Args:
        df_leyes: DataFrame con el catálogo de leyes (debe tener 'nombre', 'doc_id', 'link_docx', 'link_pdf')
    
    Returns:
        dict: {nombre_archivo: {'nombre': nombre_documento, 'doc_id': id_documento}}
    """
    mapeo = {}  # nombre_archivo -> {'nombre': nombre_documento, 'doc_id': doc_id}
    
    for idx, row in df_leyes.iterrows():
        # Preparar info del documento
        doc_info = {
            'nombre': row['nombre'],
            'doc_id': row['doc_id']
        }
        
        # Procesar link_docx
        if pd.notna(row['link_docx']):
            nombre_archivo = row['link_docx'].split('/')[-1]
            nombre_sin_ext = os.path.splitext(nombre_archivo)[0]
            # Decodificar URL y limpiar
            nombre_limpio = unquote(nombre_sin_ext)
            mapeo[nombre_limpio] = doc_info
        
        # Procesar link_pdf
        if pd.notna(row['link_pdf']):
            nombre_archivo_pdf = row['link_pdf'].split('/')[-1]
            nombre_sin_ext_pdf = os.path.splitext(nombre_archivo_pdf)[0]
            # Decodificar URL y limpiar
            nombre_limpio_pdf = unquote(nombre_sin_ext_pdf)
            if nombre_limpio_pdf not in mapeo:
                mapeo[nombre_limpio_pdf] = doc_info
    
    print(f"Mapeo creado con {len(mapeo)} documentos")
    return mapeo

def normalizar_nombre_para_match(nombre):
    """
    Normaliza un nombre de archivo para mejorar el matching.
    - Quita todos los caracteres %
    - Convierte a minúsculas para comparación insensible a mayúsculas
    """
    return nombre.replace('%', '').lower()

def crear_mapeos_manuales():
    """
    Crea mapeos manuales para archivos que no matchean bien automáticamente.
    Solo define el nombre del documento - el doc_id se obtiene del catálogo.
    
    Returns:
        dict: {nombre_archivo: nombre_documento_a_buscar}
    """
    mapeos = {
        'LEY_DE_RESP_20SOC_20MERCANTIL_DE20LA20CIUDAD20DE20MEXICO_2.4': 
            'Ley de Responsabilidad Social Mercantil de la Ciudad de Mexico'
    }
    return mapeos

def buscar_nombre_documento_fuzzy(nombre_archivo, mapeo, mapeos_manuales=None, threshold=85):
    """
    Busca el nombre del documento usando fuzzy matching.
    
    Prioridades:
    1. Match exacto
    2. Fuzzy matching (threshold 85%)
    3. Mapeo manual (solo si 1 y 2 fallan)
    
    Args:
        nombre_archivo: Nombre del archivo sin extensión
        mapeo: Diccionario con mapeo de nombres a {'nombre': ..., 'doc_id': ...}
        mapeos_manuales: Diccionario con mapeos manuales (nombre_documento -> buscar en mapeo)
        threshold: Umbral de similitud (0-100). Por defecto 85%
    
    Returns:
        tuple: (nombre_documento, doc_id) o ("SIN NOMBRE", "SIN_ID") si no hay match
    """
    if not mapeo:
        return ("SIN NOMBRE", "SIN_ID")
    
    # PRIORIDAD 1: Intentar match exacto
    if nombre_archivo in mapeo:
        doc_info = mapeo[nombre_archivo]
        return (doc_info['nombre'], doc_info['doc_id'])
    
    # PRIORIDAD 2: Fuzzy matching
    # Normalizar el nombre del archivo (quitar % y minúsculas)
    nombre_normalizado = normalizar_nombre_para_match(nombre_archivo)
    
    # Crear mapeo normalizado para fuzzy matching
    mapeo_normalizado = {
        normalizar_nombre_para_match(key): key 
        for key in mapeo.keys()
    }
    
    # Fuzzy matching con nombres normalizados
    resultado = process.extractOne(
        nombre_normalizado, 
        mapeo_normalizado.keys(), 
        scorer=fuzz.ratio,
        score_cutoff=threshold
    )
    
    if resultado:
        nombre_normalizado_match, score, _ = resultado
        nombre_original_match = mapeo_normalizado[nombre_normalizado_match]
        doc_info = mapeo[nombre_original_match]
        # Solo mostrar fuzzy matches (no exactos) para debugging
        if score < 100:
            print(f"  Fuzzy match ({score:.1f}%): '{nombre_archivo[:40]}...' -> '{nombre_original_match[:40]}...'")
        return (doc_info['nombre'], doc_info['doc_id'])
    
    # PRIORIDAD 3: Mapeo manual (solo si fuzzy matching falló)
    if mapeos_manuales and nombre_archivo in mapeos_manuales:
        nombre_buscar = mapeos_manuales[nombre_archivo]
        # Buscar en el mapeo por nombre del documento
        for key, doc_info in mapeo.items():
            if doc_info['nombre'] == nombre_buscar:
                print(f"  ✓ Match manual: '{nombre_archivo[:40]}...' -> '{nombre_buscar[:40]}...'")
                return (doc_info['nombre'], doc_info['doc_id'])
    
    return ("SIN NOMBRE", "SIN_ID")


## 1. Procesamiento de Catálogo de Leyes y Reglamentos

Cargar el catálogo completo, limpiar acentos, y crear identificadores (hashes) únicos


In [23]:
# Cargar catálogo de leyes y reglamentos
laws_regulations = pd.read_csv('/Users/alexa/Projects/cdmx_pipeline/data/01_input/leyes.csv')

print(f"Total de leyes/reglamentos: {len(laws_regulations)}")
laws_regulations.head()


Total de leyes/reglamentos: 783


,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level
0,Constitución Política de la Ciudad de México,05/02/17,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX
1,Ley de Protección de Datos Personales en Poses...,03/10/08,NaN,https://data.consejeria.cdmx.gob.mx//images/le...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX
2,Ley de Acceso de las Mujeres a una Vida Libre ...,25/06/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX
3,Ley de los Derechos de Niñas Niños y Adolescen...,02/04/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX
4,Ley para el Reconocimiento y la Atención de la...,31/03/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX


In [24]:
# Aplicar función de limpieza de acentos y hash a cada ley/reglamento

# 1. Limpiar acentos de la columna nombre (modifica in-place)
print("Limpiando acentos de nombres de documentos...")
laws_regulations['nombre'] = laws_regulations['nombre'].apply(remove_accents)

# 2. Normalizar para crear hash
laws_regulations['nombre_normalized'] = laws_regulations['nombre'].apply(normalize_text_for_hash)

# 3. Crear hash único (doc_id)
laws_regulations['doc_id'] = laws_regulations['nombre_normalized'].apply(create_document_hash)

print(f"\nProcesamiento completado:")
print(f"  - {len(laws_regulations)} leyes/reglamentos procesados")
print(f"  - {laws_regulations['doc_id'].nunique()} IDs únicos generados")
print("\nPrimeras filas:")
laws_regulations[['nombre', 'doc_id', 'link_docx', 'link_pdf']].head(10)


Limpiando acentos de nombres de documentos...

Procesamiento completado:
  - 783 leyes/reglamentos procesados
  - 783 IDs únicos generados

Primeras filas:


,nombre,doc_id,link_docx,link_pdf
0,Constitucion Politica de la Ciudad de Mexico,234F69A3,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
1,Ley de Proteccion de Datos Personales en Poses...,80360428,NaN,https://data.consejeria.cdmx.gob.mx//images/le...
2,Ley de Acceso de las Mujeres a una Vida Libre ...,74FF23BF,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
3,Ley de los Derechos de Ninas Ninos y Adolescen...,FB6CCCAE,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
4,Ley para el Reconocimiento y la Atencion de la...,6C800A8E,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
5,Ley para la Celebracion de Espectaculos Public...,298E1252,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
6,Ley Organica de Alcaldias de la Ciudad de Mexico,955199A7,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
7,Ley de Vivienda para la Ciudad De Mexico,017D8396,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
8,Ley de Seguridad Privada para la Ciudad de Mexico,71029770,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...
9,Ley de Salud de la Ciudad De Mexico,AF96B37C,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...


In [ ]:
laws_regulations['']

In [25]:
# Guardar resultados
output_path = '/Users/alexa/Projects/cdmx_pipeline/data/02_catalogs/leyes_hash.csv'
laws_regulations.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Archivo guardado en: {output_path}")


Archivo guardado en: /Users/alexa/Projects/cdmx_pipeline/data/02_catalogs/leyes_hash.csv


## 2. Procesamiento de Artículos Extraídos

Cargar los artículos extraídos, hacer fuzzy matching con el catálogo de leyes, y crear identificadores únicos para cada artículo


In [26]:
# Cargar archivo Excel con artículos extraídos
df = pd.read_excel('../../data/01_input/extracted_articles.xlsx')

print(f"Total de artículos: {len(df)}")
print(f"Documentos únicos: {df['Archivo'].nunique()}")
print(f"Columnas: {list(df.columns)}")
print("\nPrimeras filas:")
df.head()


Total de artículos: 30351
Documentos únicos: 337
Columnas: ['Archivo', 'Articulo', 'Texto']

Primeras filas:


,Archivo,Articulo,Texto
0,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 1,Las disposiciones contenidas en este Reglament...
1,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 2,La aplicación del presente Reglamento correspo...
2,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 3,"Para los efectos del presente Reglamento, adem..."
3,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 4,Sin perjuicio de los principios que prevén los...
4,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 5,El tratamiento para la persona privada de su l...


In [27]:
# Crear mapeo de nombres de archivo con el catálogo de leyes ya procesado
mapeo = crear_mapeo_nombres(laws_regulations)

# Crear mapeos manuales para casos especiales
mapeos_manuales = crear_mapeos_manuales()
print(f"Mapeos manuales creados: {len(mapeos_manuales)}")

# Hacer fuzzy matching para cada artículo extraído
print("\nMapeando artículos con catálogo de leyes...")
print(f"Prioridades: 1) Match exacto, 2) Fuzzy matching (≥85%), 3) Mapeo manual\n")
resultados_match = []

for idx, row in df.iterrows():
    nombre_archivo = row['Archivo']
    nombre_documento, doc_id = buscar_nombre_documento_fuzzy(nombre_archivo, mapeo, mapeos_manuales=mapeos_manuales, threshold=85)
    
    resultados_match.append({
        'Nombre_Documento': nombre_documento,
        'doc_id': doc_id
    })

# Agregar columnas al DataFrame
df_match = pd.DataFrame(resultados_match)
df['Nombre_Documento'] = df_match['Nombre_Documento']
df['doc_id'] = df_match['doc_id']

# # Extraer número del artículo de la columna "Articulo" para crear art_id único
# def extraer_numero_articulo(articulo_texto):
#     """Extrae el número del artículo o nombre ordinal del texto"""
#     if pd.isna(articulo_texto):
#         return "NOART"
    
#     articulo_str = str(articulo_texto).upper()
    
#     # Detectar si es TRANSITORIO/TRANSITORIOS
#     if 'TRANSITORIO' in articulo_str:
#         return "TRANS"
    
#     # Intentar extraer números primero (1, 2, 3, 117, 117bis, etc.)
#     match_num = re.search(r'(\d+(?:\s*(?:bis|ter|qu[aá]ter)?)?)', articulo_str, re.IGNORECASE)
#     if match_num:
#         return match_num.group(1).replace(' ', '')
    
#     # Si no hay números, buscar nombres ordinales (PRIMERO, SEGUNDO, etc.)
#     ordinales = [
#         'PRIMERO', 'SEGUNDO', 'TERCERO', 'CUARTO', 'QUINTO', 'SEXTO',
#         'SEPTIMO', 'SÉPTIMO', 'OCTAVO', 'NOVENO', 'DECIMO', 'DÉCIMO',
#         'UNDECIMO', 'UNDÉCIMO', 'DUODECIMO', 'DUODÉCIMO'
#     ]
    
#     for ordinal in ordinales:
#         if ordinal in articulo_str:
#             # Retornar el ordinal sin acentos
#             return ordinal.replace('É', 'E').replace('Á', 'A').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    
#     return "NOART"

def extraer_numero_articulo(articulo_texto):
    """
    Extrae el número del artículo con todos sus sufijos para crear art_id único.
    
    Ejemplos de retorno:
    - "Artículo 449" → "449"
    - "Artículo 449 Bis" → "449BIS"
    - "Artículo 498 Bis 6" → "498BIS6"
    - "Artículo 272 A" → "272A"
    - "Artículo 8o-A" → "8A"
    - "Artículo 23-1" → "23-1"
    - "Artículo Primero" → "PRIMERO"
    - "Transitorio" → "TRANS"
    """
    if pd.isna(articulo_texto):
        return "NOART"
    
    articulo_str = str(articulo_texto).upper()
    
    # Detectar si es TRANSITORIO/TRANSITORIOS
    if 'TRANSITORIO' in articulo_str:
        # Intentar extraer ordinal transitorio
        ordinales_trans = [
            'PRIMERO', 'SEGUNDO', 'TERCERO', 'CUARTO', 'QUINTO', 'SEXTO',
            'SEPTIMO', 'SÉPTIMO', 'OCTAVO', 'NOVENO', 'DECIMO', 'DÉCIMO'
        ]
        for ordinal in ordinales_trans:
            if ordinal in articulo_str:
                return f"TRANS_{ordinal.replace('É', 'E').replace('Á', 'A')}"
        return "TRANS"
    
    # Patrón completo para capturar número base + todos los sufijos posibles
    # Captura: número + [letra con/sin guión OR fracción OR ordinal latino + opcional número]
    patron = r'(\d+)[°ºªo]?\s*(?:' \
             r'([A-Z])(?!\w)|' \
             r'-([A-Z])(?!\w)|' \
             r'-(\d+)|' \
             r'(BIS|TER|QU[AÁ]TER|QUINTUS|QUINQUIES|QUINTIES|' \
             r'SEXTUS|SEXIES|SEXTIES|SEPTIMUS|SEPTIES|SEPTIESEP|' \
             r'OCTAVUS|OCTIES|NONIES|NON[IÍ]ES|NOVIES|' \
             r'DECIES|DEC[IÍ]ES|DECIMUS|UNDECIES|DUODECIES)' \
             r'(?:\s+(\d+))?' \
             r')?'
    
    match = re.search(patron, articulo_str, re.IGNORECASE)
    
    if match:
        numero_base = match.group(1)
        letra_espacio = match.group(2)  # Letra con espacio: "272 A"
        letra_guion = match.group(3)     # Letra con guión: "8o-A"
        fraccion = match.group(4)        # Fracción: "23-1"
        ordinal = match.group(5)         # Ordinal latino: "BIS", "TER", etc.
        num_ordinal = match.group(6)    # Número después de ordinal: "BIS 6"
        
        # Construir el identificador
        resultado = numero_base
        
        # Agregar letra (con o sin guión)
        if letra_espacio:
            resultado += letra_espacio
        elif letra_guion:
            resultado += letra_guion
        # Agregar fracción
        elif fraccion:
            resultado += f"-{fraccion}"
        # Agregar ordinal latino
        elif ordinal:
            # Normalizar ordinales comunes
            ordinal_normalizado = ordinal.upper()
            ordinal_normalizado = ordinal_normalizado.replace('Á', 'A').replace('Í', 'I')
            resultado += ordinal_normalizado
            
            # Agregar número después del ordinal si existe
            if num_ordinal:
                resultado += num_ordinal
        
        return resultado
    
    # Si no hay números, buscar nombres ordinales (PRIMERO, SEGUNDO, etc.)
    ordinales = [
        'PRIMERO', 'SEGUNDO', 'TERCERO', 'CUARTO', 'QUINTO', 'SEXTO',
        'SEPTIMO', 'SÉPTIMO', 'OCTAVO', 'NOVENO', 'DECIMO', 'DÉCIMO',
        'UNDECIMO', 'UNDÉCIMO', 'DUODECIMO', 'DUODÉCIMO', 'DECIMOTERCERO',
        'DECIMOCUARTO', 'DECIMOQUINTO'
    ]
    
    for ordinal in ordinales:
        if ordinal in articulo_str:
            # Retornar el ordinal sin acentos
            return ordinal.replace('É', 'E').replace('Á', 'A').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    
    return "NOART"


df['art_num'] = df['Articulo'].apply(extraer_numero_articulo)

# Crear art_id único combinando doc_id y número de artículo
df['art_id'] = df.apply(
    lambda row: f"{row['doc_id']}_{row['art_num']}" if row['doc_id'] != 'SIN_ID' else f"UNKNOWN_{row['art_num']}", 
    axis=1
).str.replace(' ', '').str.upper()

# Reorganizar columnas
df = df[['art_id', 'doc_id', 'Archivo', 'Nombre_Documento', 'Articulo', 'art_num', 'Texto']]
df.rename(columns={'Texto': 'text'}, inplace=True)
df.rename(columns={'Nombre_Documento': 'document_name'}, inplace=True)
df.rename(columns={'Articulo': 'article_name'}, inplace=True)

# Estadísticas
print(f"\n{'='*70}")
print(f"RESUMEN DE MATCHING")
print(f"{'='*70}")
print(f"Total de artículos procesados: {len(df)}")
print(f"Documentos únicos (archivos): {df['Archivo'].nunique()}")
print(f"Documentos con match exitoso: {len(df[df['doc_id'] != 'SIN_ID'])}")
print(f"Documentos sin match: {len(df[df['doc_id'] == 'SIN_ID'])}")

# Mostrar archivos sin match
sin_match = df[df['doc_id'] == 'SIN_ID']['Archivo'].unique()
if len(sin_match) > 0:
    print(f"\nArchivos sin match ({len(sin_match)}):")
    for archivo in sin_match[:10]:
        print(f"  - {archivo}")

print(f"\nPrimeras filas procesadas:")
df.head(10)


Mapeo creado con 347 documentos
Mapeos manuales creados: 1

Mapeando artículos con catálogo de leyes...
Prioridades: 1) Match exacto, 2) Fuzzy matching (≥85%), 3) Mapeo manual

  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE...'
  Fuzzy match (97.8%): 'REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_D...' -> 'REGLAMENTO_LEY_PROTECCION _A_LA_SALUD_DE..

,art_id,doc_id,Archivo,document_name,article_name,art_num,text
0,6D0C4493_1,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 1,1,Las disposiciones contenidas en este Reglament...
1,6D0C4493_2,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 2,2,La aplicación del presente Reglamento correspo...
2,6D0C4493_3,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 3,3,"Para los efectos del presente Reglamento, adem..."
3,6D0C4493_4,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 4,4,Sin perjuicio de los principios que prevén los...
4,6D0C4493_5,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 5,5,El tratamiento para la persona privada de su l...
5,6D0C4493_6,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 6,6,Durante la ejecución de la pena privativa de l...
6,6D0C4493_7,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 7,7,Se prohíbe toda conducta que implique el uso d...
7,6D0C4493_8,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 8,8,El internamiento en los Centros Penitenciarios...
8,6D0C4493_9,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 9,9,Las inspecciones y revisiones que se practique...
9,6D0C4493_10,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 10,10,En los Centros Penitenciarios se brindará la a...


In [28]:
# Existen art_id duplicados?
duplicados = df[df.duplicated(subset=['art_id'])]
print(duplicados['doc_id'].unique())
print(duplicados['doc_id'].nunique())


['EDEA48F4' 'C57B66D6' 'BD22E724' '2F07FC1B' 'D0D23E5F' '1C73B492'
 'FC3F5431' 'B848E072' '5439F880' '62CDA32A' '35C270ED' 'CE7E985D']
12


In [29]:
# Guardar resultados con identificadores
output_path = '../../data/03_extracted/legal_documents_with_ids.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\nArchivo guardado en: {output_path}")
print(f"Total de registros guardados: {len(df)}")



Archivo guardado en: ../../data/03_extracted/legal_documents_with_ids.csv
Total de registros guardados: 30351


## 3. Procesamiento de Entes Públicos

Crear identificadores para el catálogo de entes públicos


In [30]:
# Cargar catálogo de entes públicos
entes_publicos = pd.read_csv('../../data/02_catalogs/entes-publicos.csv')

print(f"Total de entes públicos: {len(entes_publicos)}")
entes_publicos.head()


Total de entes públicos: 112


,unidad_responsable,nombre,gobierno_general,desc_gobierno_general,ambito,tipo_administracion,acronimo
0,01C001,JEFATURA DE GOBIERNO,1,PODER EJECUTIVO,CENTRAL,UNIDAD ADMINISTRATIVA,JG
1,01CD03,"CENTRO DE COMANDO, CONTROL, CÓMPUTO, COMUNICAC...",1,PODER EJECUTIVO,DESCONCENTRADO,ÓRGANO,C5
2,01CD06,AGENCIA DIGITAL DE INNOVACION PUBLICA,1,PODER EJECUTIVO,DESCONCENTRADO,DEPENDENCIA,ADIP
3,02C001,SECRETARÍA DE GOBIERNO,2,PODER EJECUTIVO,CENTRAL,DEPENDENCIA,SEGOBCDMX
4,02CD01,ALCALDÍA ÁLVARO OBREGÓN,2,PODER EJECUTIVO,GOBIERNO LOCAL,ALCALDÍA,AO


In [31]:
# Aplicar función de hash a cada ente público
entes_publicos['nombre'] = entes_publicos['nombre'].apply(remove_accents)
entes_publicos['name_nomalized'] = entes_publicos['nombre'].apply(normalize_text_for_hash)
entes_publicos['entity_id'] = entes_publicos['name_nomalized'].apply(create_document_hash)

print("\nPrimeras filas con identificadores:")
entes_publicos.head()



Primeras filas con identificadores:


,unidad_responsable,nombre,gobierno_general,desc_gobierno_general,ambito,tipo_administracion,acronimo,name_nomalized,entity_id
0,01C001,JEFATURA DE GOBIERNO,1,PODER EJECUTIVO,CENTRAL,UNIDAD ADMINISTRATIVA,JG,JEFATURADEGOBIERNO,E2EEE4C1
1,01CD03,"CENTRO DE COMANDO, CONTROL, COMPUTO, COMUNICAC...",1,PODER EJECUTIVO,DESCONCENTRADO,ÓRGANO,C5,CENTRODECOMANDOCONTROLCOMPUTOCOMUNICACIONESYCO...,50B56830
2,01CD06,AGENCIA DIGITAL DE INNOVACION PUBLICA,1,PODER EJECUTIVO,DESCONCENTRADO,DEPENDENCIA,ADIP,AGENCIADIGITALDEINNOVACIONPUBLICA,11A7BB35
3,02C001,SECRETARIA DE GOBIERNO,2,PODER EJECUTIVO,CENTRAL,DEPENDENCIA,SEGOBCDMX,SECRETARIADEGOBIERNO,919DEC37
4,02CD01,ALCALDIA ALVARO OBREGON,2,PODER EJECUTIVO,GOBIERNO LOCAL,ALCALDÍA,AO,ALCALDIAALVAROOBREGON,FD7B16D9


In [32]:
# Guardar resultados
output_path = '../../data/02_catalogs/entes-publicos_hash.csv'
entes_publicos.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Archivo guardado en: {output_path}")


Archivo guardado en: ../../data/02_catalogs/entes-publicos_hash.csv
